# **<font color="red">Context</font>**

In [1]:
import os
from config import config
os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

In [2]:
from google.adk.agents import Agent
from google.adk.tools import FunctionTool
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

APP_NAME="weather_sentiment_agent"
USER_ID="user1234"
SESSION_ID="1234"
MODEL_ID="gemini-2.5-flash"

# Tool 1
def get_weather_report(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Returns:
        dict: A dictionary containing the weather information with a 'status' key ('success' or 'error') and a 'report' key with the weather details if successful, or an 'error_message' if an error occurred.
    """
    if city.lower() == "london":
        return {"status": "success", "report": "The current weather in London is cloudy with a temperature of 18 degrees Celsius and a chance of rain."}
    elif city.lower() == "paris":
        return {"status": "success", "report": "The weather in Paris is sunny with a temperature of 25 degrees Celsius."}
    else:
        return {"status": "error", "error_message": f"Weather information for '{city}' is not available."}

weather_tool = FunctionTool(func=get_weather_report)


# Tool 2
def analyze_sentiment(text: str) -> dict:
    """Analyzes the sentiment of the given text.

    Returns:
        dict: A dictionary with 'sentiment' ('positive', 'negative', or 'neutral') and a 'confidence' score.
    """
    if "good" in text.lower() or "sunny" in text.lower():
        return {"sentiment": "positive", "confidence": 0.8}
    elif "rain" in text.lower() or "bad" in text.lower():
        return {"sentiment": "negative", "confidence": 0.7}
    else:
        return {"sentiment": "neutral", "confidence": 0.6}

sentiment_tool = FunctionTool(func=analyze_sentiment)


# Agent
weather_sentiment_agent = Agent(
    model=MODEL_ID,
    name='weather_sentiment_agent',
    instruction="""You are a helpful assistant that provides weather information and analyzes the sentiment of user feedback.
**If the user asks about the weather in a specific city, use the 'get_weather_report' tool to retrieve the weather details.**
**If the 'get_weather_report' tool returns a 'success' status, provide the weather report to the user.**
**If the 'get_weather_report' tool returns an 'error' status, inform the user that the weather information for the specified city is not available and ask if they have another city in mind.**
**After providing a weather report, if the user gives feedback on the weather (e.g., 'That's good' or 'I don't like rain'), use the 'analyze_sentiment' tool to understand their sentiment.** Then, briefly acknowledge their sentiment.
You can handle these tasks sequentially if needed.""",
    tools=[weather_tool, sentiment_tool]
)


"""Main function to run the agent asynchronously."""
# Session and Runner Setup
session_service = InMemorySessionService()
# Use 'await' to correctly create the session
await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)

runner = Runner(agent=weather_sentiment_agent, app_name=APP_NAME, session_service=session_service)

# Agent Interaction
query = "weather in london?"
print(f"User Query: {query}")
content = types.Content(role='user', parts=[types.Part(text=query)])

# The runner's run method handles the async loop internally
events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

for event in events:
    if event.is_final_response():
        final_response = event.content.parts[0].text
        print("Agent Response:", final_response)



User Query: weather in london?


D:\Agent-Development-Kit\venv\Lib\site-packages\google\adk\flows\llm_flows\base_llm_flow.py:449: UserWarning: [EXPERIMENTAL] feature FeatureName.PROGRESSIVE_SSE_STREAMING is enabled.
  async for event in agen:


Agent Response: The current weather in London is cloudy with a temperature of 18 degrees Celsius and a chance of rain.


## **Context:** 
- In ADK context refers to the crucial bundle of information available to your agent and its tools during specific operations.
- Agents often need more that just the latest user message to perform well. Context is essential because it enables:
  1. **Maintaining State:** Remember details across multiple steps in a conversation. This is primarily managed through _session state_.
  2. **Passing Data:** Shared information discoverd or generated in one step(like an LLM call or a tool execution) with subsequent steps. Session State is key here too.
  3. **Accessing Services:** Interacting with framewok capabilities like:
     - **Artifact Storage:** Saving or loading files or data blobs (like PDFs, images, configuration files) associated with the session.
     - **Memory:** Searching for relevant information from past interactions or external knowledge sources conected to the user.
     - **Authentication:** Requesting and retrieving credentials needed by tools to access external APIs security.
  4. **Identity and Tracking:** Knowing which agent is currently running (`agent.name`) and uniquely identifying the current request-response cycle (`invocation_id`) for logging and debugging.
  5. **Tool-Specific Actions:** Enabling specialized operations withing tools, such as requesting authenticaiton or searching memory, which require access to the current interaction's details.
- The central piece holding all this information together for a single, complete user-request-to-final-response cycle (an **invocation**) is the `InvocationContext`. However, you typically won't create or manage this object directly. The ADK framework creates it when an invocation starts (e.g., via `runner.run_async`) and passes the relevant contextual information implicitly to your agent code, callbacks, and tools.

### **Types of Context:**
1. `InvocationContext`
2. `ReadonlyContext`
3. `CallbackContext`
4. `ToolContext`

### **<font color="blue">InvocationContext</font>**
- **Where Used:** Received as the ctx argument directly within an agent's core implementation methods (`_run_async_impl`, `_run_live_impl`).
- **Purpose:** Provides access to the entire state of the current invocation. This is the most comprehensive context object.
- **Key Contents:** Direct access to `session` (including `state` and `events`), the current `agent` instance, `invocation_id`, initial `user_content`, references to configured services (`artifact_service`, `memory_service`, `session_service`), and fields related to live/streaming modes.
**Use Case:** Primarily used when the agent's core logic needs direct access to the overall session or services, though often state and artifact interactions are delegated to callbacks/tools which use their own contexts. Also used to control the invocation itself (e.g., setting `ctx.end_invocation = True`).

In [3]:
# Pseudocode: Agent implementation receiving InvocationContext
from google.adk.agents import BaseAgent
from google.adk.agents.invocation_context import InvocationContext
from google.adk.events import Event
from typing import AsyncGenerator

class MyAgent(BaseAgent):
    async def _run_async_impl(self, ctx: InvocationContext) -> AsyncGenerator[Event, None]:
        # Direct access example
        agent_name = ctx.agent.name
        session_id = ctx.session.id
        print(f"Agent {agent_name} running in session {session_id} for invocation {ctx.invocation_id}")
        # ... agent logic using ctx ...
        yield # ... event ...

In [4]:
import os
import asyncio
from typing import AsyncGenerator

from google.adk.agents import BaseAgent
from google.adk.agents.invocation_context import InvocationContext
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.events import Event
from google.genai import types

from config import config

# -------------------------------------------------
# Configuration
# -------------------------------------------------
os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "invocation_context_app"
USER_ID = "user_1"
SESSION_ID = "session_1"


# -------------------------------------------------
# Custom Agent Using InvocationContext
# -------------------------------------------------
class MyAgent(BaseAgent):

    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:

        # Access InvocationContext data
        agent_name = ctx.agent.name
        session_id = ctx.session.id
        invocation_id = ctx.invocation_id
        user_id = ctx.session.user_id

        print("=" * 60)
        print(f"Agent Name     : {agent_name}")
        print(f"Session ID     : {session_id}")
        print(f"User ID        : {user_id}")
        print(f"Invocation ID  : {invocation_id}")
        print("=" * 60)
        
        # Create response
        response_text = (
            f"Hello {user_id}!\n"
            f"This was processed in invocation: {invocation_id}"
        )

        # Yield response event
        yield Event(
            author=self.name,
            content=types.Content(
                role="model",
                parts=[types.Part(text=response_text)],
            ),
        )


# -------------------------------------------------
# Main Runner Logic
# -------------------------------------------------

# Create session service
session_service = InMemorySessionService()

# Create session
await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

# Create agent
agent = MyAgent(name="ContextAwareAgent")

# Create runner
runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
)

# Send a user message
print("\nSending user message...\n")

async for event in runner.run_async(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=types.Content(
        role="user",
        parts=[types.Part(text="Hello Agent!")],
    ),
):
    if event.content and event.content.parts:
        print("\nAgent Response:")
        print(event.content.parts[0].text)




Sending user message...

Agent Name     : ContextAwareAgent
Session ID     : session_1
User ID        : user_1
Invocation ID  : e-b320a9e5-27c6-488b-a253-7c7afa3b4923

Agent Response:
Hello user_1!
This was processed in invocation: e-b320a9e5-27c6-488b-a253-7c7afa3b4923


### **<font color="blue">ReadonlyContext</font>**
- **Where Used:** Provided in scenarios where only read access to basic information is needed and mutation is disallowed (e.g., `InstructionProvider` functions). It's also the base class for other contexts.
- **Purpose:** Offers a safe, read-only view of fundamental contextual details.
- **Key Contents:** `invocation_id`, `agent_name`, and a read-only view of the current `state`.

In [5]:
# Pseudocode: Instruction provider receiving ReadonlyContext
from google.adk.agents.readonly_context import ReadonlyContext

def my_instruction_provider(context: ReadonlyContext) -> str:
    # Read-only access example
    user_tier = context.state().get("user_tier", "standard") # Can read state
    # context.state['new_key'] = 'value' # This would typically cause an error or be ineffective
    return f"Process the request for a {user_tier} user."

In [6]:
import os
import asyncio

from google.adk.agents import LlmAgent
from google.adk.agents.readonly_context import ReadonlyContext
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

from config import config

# -------------------------------------------------
# Configuration
# -------------------------------------------------
os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "readonly_context_app"
USER_ID = "user_1"
SESSION_ID = "session_1"


# -------------------------------------------------
# Dynamic Instruction Provider
# -------------------------------------------------
def my_instruction_provider(context: ReadonlyContext) -> str:
    # Read session state safely
    user_tier = context.state.get("user_tier", "standard")

    return f"""
You are a helpful AI assistant.
Process the request for a {user_tier} user.
If the user is premium, provide detailed and priority responses.
If standard, provide concise helpful answers.
"""


# -------------------------------------------------
# Main Execution
# -------------------------------------------------

# Create session service
session_service = InMemorySessionService()

# Create session with initial state
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
    state={"user_tier": "premium"},   # Change to "standard" to test
)

# Create LLM Agent with dynamic instruction provider
agent = LlmAgent(
    name="TierAwareAgent",
    model="gemini-2.5-flash",
    instruction=my_instruction_provider,  # Using ReadonlyContext here
)

# Create runner
runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
)

print("\nSending message...\n")

# Run agent
async for event in runner.run_async(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=types.Content(
        role="user",
        parts=[types.Part(text="Explain what AI agents are.")],
    ),
):
    if event.content and event.content.parts:
        print("Agent Response:\n")
        print(event.content.parts[0].text)




Sending message...

Agent Response:

As a premium user, you'll receive a detailed and comprehensive explanation of AI agents.

---

### What are AI Agents?

At its core, an **AI agent** is anything that can perceive its environment through sensors and act upon that environment through effectors. It's a system designed to operate autonomously, pursuing specific goals, and often interacting with other agents or humans.

Think of an AI agent as an intelligent entity that observes, thinks, and acts within a particular context. This definition is quite broad, encompassing everything from a simple thermostat to a complex self-driving car or a sophisticated large language model.

### Core Components and Characteristics:

To better understand AI agents, let's break down their fundamental components and characteristics:

1.  **Environment:** This is the world in which the agent exists and operates. It can be physical (e.g., a room for a robot) or digital (e.g., the internet for a web crawler, 

### **<font color="blue">CallbackContext</font>**
- **Where Used:** Passed as `callback_context` to agent lifecycle callbacks (`before_agent_callback`, `after_agent_callback`) and model interaction callbacks (`before_model_callback`, `after_model_callback`).
- **Purpose:** Facilitates inspecting and modifying state, interacting with artifacts, and accessing invocation details specifically within callbacks.
- **Key Capabilities (Adds to `ReadonlyContext`):**
  - **Mutable `state` Property:** Allows reading and writing to session state. Changes made here (`callback_context.state['key'] = value`) are tracked and associated with the event generated by the framework after the callback.
  - **Artifact Methods:** `load_artifact(filename)` and `save_artifact(filename, part)` methods for interacting with the configured `artifact_service`.
- Direct `user_content` access.


In [7]:
# Pseudocode: Callback receiving CallbackContext
from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmRequest
from google.genai import types
from typing import Optional

def my_before_model_cb(callback_context: CallbackContext, request: LlmRequest) -> Optional[types.Content]:
    # Read/Write state example
    call_count = callback_context.state.get("model_calls", 0)
    callback_context.state["model_calls"] = call_count + 1 # Modify state

    # Optionally load an artifact
    # config_part = callback_context.load_artifact("model_config.json")
    print(f"Preparing model call #{call_count + 1} for invocation {callback_context.invocation_id}")
    return None # Allow model call to proceed

In [8]:
import os
import asyncio
from typing import Optional

from google.adk.agents import LlmAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmRequest
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

from config import config

# -------------------------------------------------
# Configuration
# -------------------------------------------------
os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "callback_context_app"
USER_ID = "user_1"
SESSION_ID = "session_1"


# -------------------------------------------------
# BEFORE MODEL CALLBACK (Correct Signature)
# -------------------------------------------------
def my_before_model_cb(
    callback_context: CallbackContext,
    llm_request: LlmRequest,
) -> Optional[types.Content]:

    # ✅ Read state
    call_count = callback_context.state.get("model_calls", 0)

    # ✅ Write state
    callback_context.state["model_calls"] = call_count + 1

    print("=" * 60)
    print(f"Model Call #: {call_count + 1}")
    print(f"Invocation ID: {callback_context.invocation_id}")
    print(f"Agent Name   : {callback_context.agent_name}")
    print("=" * 60)

    # ✅ Direct user content access
    if callback_context.user_content:
        print("User said:", callback_context.user_content.parts[0].text)

    # Returning None allows model call to proceed
    return None


# -------------------------------------------------
# Main Execution
# -------------------------------------------------
session_service = InMemorySessionService()

await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
    state={"model_calls": 0},
)

agent = LlmAgent(
    name="CallbackAgent",
    model="gemini-2.5-flash",
    instruction="You are a helpful assistant.",
    before_model_callback=my_before_model_cb,
)

runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
)

print("\nSending message...\n")

async for event in runner.run_async(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=types.Content(
        role="user",
        parts=[types.Part(text="What is an AI agent?")],
    ),
):
    if event.content and event.content.parts:
        print("\nAgent Response:\n")
        print(event.content.parts[0].text)




Sending message...

Model Call #: 1
Invocation ID: e-28580363-b6f6-4b56-bafe-9b4c6498c175
Agent Name   : CallbackAgent
User said: What is an AI agent?

Agent Response:

An **AI agent** is a fundamental concept in artificial intelligence, representing an entity that perceives its environment through sensors and acts upon that environment through actuators.

Essentially, an AI agent is designed to achieve specific goals or objectives within a given environment.

Here's a breakdown of its key characteristics:

1.  **Perception:** Agents gather information from their environment using "sensors." This could be anything from a camera (for a robot), a microphone (for a voice assistant), a text input (for a chatbot), or data feeds (for a trading bot).
2.  **Action:** Agents perform actions that affect their environment using "actuators." Examples include moving a robotic arm, speaking a reply, sending a message, or making a trade.
3.  **Environment:** This is the context in which the agent op

### **<font color="blue">ToolContext</font>**
- **Where Used:** Passed as `tool_context` to the functions backing `FunctionTools` and to tool execution callbacks (`before_tool_callback`, `after_tool_callback`).
- **Purpose:** Provides everything `CallbackContext` does, plus specialized methods essential for tool execution, like handling authentication, searching memory, and listing artifacts.
- **Key Capabilities (Adds to `CallbackContext`):**
  - **Authentication Methods:** `request_credential(auth_config)` to trigger an auth flow, and `get_auth_response(auth_config)` to retrieve credentials provided by the user/system.
  - **Artifact Listing:** `list_artifacts()` to discover available artifacts in the session.
  - **Memory Search:** `search_memory(query)` to query the configured `memory_service`.
  - **`function_call_id` Property:** Identifies the specific function call from the LLM that triggered this tool execution, crucial for linking authentication requests or responses back correctly.
  - **`actions` Property:** Direct access to the `EventActions` object for this step, allowing the tool to signal state changes, auth requests, etc.

In [9]:
# Pseudocode: Tool function receiving ToolContext
from google.adk.tools import ToolContext
from typing import Dict, Any

# Assume this function is wrapped by a FunctionTool
def search_external_api(query: str, tool_context: ToolContext) -> Dict[str, Any]:
    api_key = tool_context.state.get("api_key")
    if not api_key:
        # Define required auth config
        # auth_config = AuthConfig(...)
        # tool_context.request_credential(auth_config) # Request credentials
        # Use the 'actions' property to signal the auth request has been made
        # tool_context.actions.requested_auth_configs[tool_context.function_call_id] = auth_config
        return {"status": "Auth Required"}

    # Use the API key...
    print(f"Tool executing for query '{query}' using API key. Invocation: {tool_context.invocation_id}")

    # Optionally search memory or list artifacts
    # relevant_docs = tool_context.search_memory(f"info related to {query}")
    # available_files = tool_context.list_artifacts()

    return {"result": f"Data for {query} fetched."}

In [10]:
import os
import asyncio
from typing import Dict, Any

from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import FunctionTool, ToolContext
from google.adk.auth import AuthConfig
from google.genai import types

from config import config

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "enterprise_auth_app"
USER_ID = "user_1"
SESSION_ID = "session_1"


# -------------------------------------------------
# Tool Function
# -------------------------------------------------
def search_external_api(
    query: str,
    tool_context: ToolContext
) -> Dict[str, Any]:
    """
    Search secure external API data using a query string.
    Requires an API key.
    """

    api_key = tool_context.state.get("external_api_key")

    # If no API key → trigger ADK Credential Request
    if not api_key:
        auth_config = AuthConfig(
            authScheme={
                "type": "apiKey",
                "in": "header",
                "name": "X-API-Key"
            },
            description="Please provide your External API Key."
        )

        tool_context.request_credential(auth_config)

        return {"status": "Authentication Required"}

    # If API key exists
    print("=" * 60)
    print(f"Using API Key: {api_key}")
    print(f"Invocation ID: {tool_context.invocation_id}")
    print("=" * 60)

    return {
        "result": f"Secure data fetched for query '{query}'."
    }


# -------------------------------------------------
# Main Async Runner
# -------------------------------------------------
session_service = InMemorySessionService()

await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

search_tool = FunctionTool(search_external_api)

agent = LlmAgent(
    name="EnterpriseAgent",
    model="gemini-2.5-flash",
    instruction="""
If user asks to search secure data,
call the search_external_api tool.
""",
    tools=[search_tool],
)

runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
)

async for event in runner.run_async(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=types.Content(
        role="user",
        parts=[types.Part(text="Search financial market data")],
    ),
):
    if event.content and event.content.parts:
        print("\nAgent Response:\n")
        print(event.content.parts[0].text)




Agent Response:

None

Agent Response:

None

Agent Response:

None

Agent Response:

I'm sorry, I cannot fulfill this request. The financial market data is secure and requires an API key to access. It seems that the API key is missing or invalid.


## **<font color="red">Configure context caching</font>**
- You configure the context caching feature at the ADK `App` object level, which wraps your agent. Use the `ContextCacheConfig` class to configure these settings, as shown in the following code sample.

In [11]:
import os
import asyncio

from google.adk import Agent
from google.adk.apps.app import App
from google.adk.agents.context_cache_config import ContextCacheConfig
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

from config import config

# -------------------------------------------------
# Configuration
# -------------------------------------------------
os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "my_caching_agent_app"
USER_ID = "user_1"
SESSION_ID = "session_1"


# -------------------------------------------------
# 1️⃣ Create Root Agent
# -------------------------------------------------
root_agent = Agent(
    name="CachingAgent",
    model="gemini-2.5-flash",
    instruction="""
You are a helpful AI assistant.
Provide detailed explanations when appropriate.
""",
)


# -------------------------------------------------
# 2️⃣ Create App with Context Caching
# -------------------------------------------------
app = App(
    name=APP_NAME,   # Must be valid identifier
    root_agent=root_agent,
    context_cache_config=ContextCacheConfig(
        min_tokens=2048,
        ttl_seconds=600,
        cache_intervals=5,
    ),
)


# -------------------------------------------------
# 3️⃣ Run the App
# -------------------------------------------------
session_service = InMemorySessionService()

await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

runner = Runner(
    app=app,  # ⚠ Important: pass App
    session_service=session_service,
)

print("\nSending message...\n")

async for event in runner.run_async(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=types.Content(
        role="user",
        parts=[types.Part(
            text="Explain in detail how transformers work."
        )],
    ),
):
    if event.content and event.content.parts:
        print("\nAgent Response:\n")
        print(event.content.parts[0].text)




Sending message...



C:\Users\DELL\AppData\Local\Temp\ipykernel_14056\3186723127.py:42: UserWarning: [EXPERIMENTAL] ContextCacheConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  context_cache_config=ContextCacheConfig(
D:\Agent-Development-Kit\venv\Lib\site-packages\google\adk\models\google_llm.py:172: UserWarning: [EXPERIMENTAL] GeminiContextCacheManager: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  cache_manager = GeminiContextCacheManager(self.api_client)



Agent Response:

The Transformer architecture, introduced in the 2017 paper "Attention Is All You Need" by Vaswani et al., revolutionized the field of natural language processing (NLP) and sequence modeling. Before Transformers, recurrent neural networks (RNNs) and their variants (LSTMs, GRUs) were dominant for sequence tasks. Transformers achieved superior performance by replacing recurrence with an **attention mechanism**, allowing for highly parallel computation and better handling of long-range dependencies.

Let's break down how Transformers work in detail.

---

### I. The Core Idea: Attention

The fundamental innovation of the Transformer is the **attention mechanism**. Instead of processing a sequence word by word (like RNNs), attention allows the model to weigh the importance of different words in the input sequence when processing each word.

Imagine you're translating the sentence "The animal didn't cross the street because it was too tired." When deciding what "it" refers 

## **<font color='red'>Compress agent context for performance</font>**
- As an ADK agent runs it collects _context_ information, including user instructions, retrieved data, tool responses, and generated content. As the size of this context data grows, agent processing times typically also increase. More and more data is sent to the generative AI model used by the agent, increasing processing time and slowing down responses. The ADK Context Compaction feature is designed to reduce the size of context as an agent is running by summarizing older parts of the agent workflow event history.
- The Context Compaction feature uses a sliding window approach for collecting and summarizing agent workflow event data within a __Session__. When you configure this feature in your agent, it summarizes data from older events once it reaches a threshold of a specific number of workflow events, or invocations, with the current Session.
- Add context compaction to your agent workflow by adding an Events Compaction Configuration setting to the App object of your workflow. As part of the configuration, you must specify a compaction interval and overlap size, as shown in the following sample code:

In [12]:
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.apps.llm_event_summarizer import LlmEventSummarizer
from google.adk.models import Gemini

# Define the AI model to be used for summarization:
summarization_llm = Gemini(model="gemini-2.5-flash")

# Create the summarizer with the custom model:
my_summarizer = LlmEventSummarizer(llm=summarization_llm)

# Configure the App with the custom summarizer and compaction settings:
app = App(
    name=APP_NAME,
    root_agent=root_agent,
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,
        overlap_size=1,
        summarizer=my_summarizer,
    ),
)


C:\Users\DELL\AppData\Local\Temp\ipykernel_14056\3827632384.py:15: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


In [13]:
import os
import asyncio

from google.adk import Agent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.apps.llm_event_summarizer import LlmEventSummarizer
from google.adk.models import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

from config import config

# -------------------------------------------------
# Configuration
# -------------------------------------------------
os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "my_agent_app"   # must be valid identifier
USER_ID = "user_1"
SESSION_ID = "session_1"


# -------------------------------------------------
# 1️⃣ Root Agent (Main Conversational Model)
# -------------------------------------------------
root_agent = Agent(
    name="MainAgent",
    model="gemini-2.0-flash",
    instruction="""
You are a helpful AI assistant.
Provide clear and structured explanations.
""",
)


# -------------------------------------------------
# 2️⃣ Summarization Model (Used for Compaction)
# -------------------------------------------------
summarization_llm = Gemini(
    model="gemini-2.5-flash"   # Used ONLY for summarization
)

my_summarizer = LlmEventSummarizer(
    llm=summarization_llm
)


# -------------------------------------------------
# 3️⃣ Create App with Events Compaction Config
# -------------------------------------------------
app = App(
    name=APP_NAME,
    root_agent=root_agent,
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # Compact after every 3 events
        overlap_size=1,         # Keep last 1 event before summarizing
        summarizer=my_summarizer,
    ),
)


# -------------------------------------------------
# 4️⃣ Run the App
# -------------------------------------------------
session_service = InMemorySessionService()

# Create session
await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

runner = Runner(
    app=app,   # Important: pass App, not Agent
    session_service=session_service,
)

print("Starting Conversation...\n")

user_messages = [
    "Explain what AI agents are.",
    "How do they use memory?",
    "What is event compaction?",
    "Why is summarization needed?",
    "Explain transformers briefly."
]

for msg in user_messages:
    print("="*100)
    print(f"User: {msg}\n")

    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=types.Content(
            role="user",
            parts=[types.Part(text=msg)],
        ),
    ):
        if event.content and event.content.parts:
            print("Agent:")
            print(event.content.parts[0].text)
            print("="*100)


Starting Conversation...

User: Explain what AI agents are.



C:\Users\DELL\AppData\Local\Temp\ipykernel_14056\193324168.py:55: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


Agent:
Okay, let's break down AI Agents.

**What is an AI Agent?**

At its core, an AI agent is an autonomous entity that perceives its environment through sensors and acts upon that environment through actuators to achieve a specific goal. Think of it as a computer program that can make decisions and take actions independently.

Here's a more detailed breakdown of the key aspects:

*   **Autonomy:** AI agents operate without direct, continuous human intervention. They can make decisions and take actions on their own, based on their programming and the data they receive.
*   **Perception:** They perceive their environment using sensors. These sensors can be anything from cameras and microphones to data feeds and APIs. The agent uses this sensory input to understand the current state of its environment.
*   **Action:** AI agents act upon their environment through actuators. Actuators are the mechanisms that allow the agent to perform actions, such as moving a robot arm, sending an email